In [95]:
import os
from glob import glob

import pandas as pd
import numpy as np

from sklearn.linear_model import RidgeCV

In [96]:
DATA_FOLDER = '/home/shashanka-subrahmanya/epic-2023-challenge/data/'
RESULT_FOLDER = '/home/shashanka-subrahmanya/epic-2023-challenge/results/'

# Scenario 1

In [97]:
def concat(path):
    
    fnames = glob(os.path.join(path, '*'))
    X, Y = [], []
    for name in fnames:
    
        annotation = name.split('/')
        annotation[-2] = 'annotations'
        annotation = '/'.join(annotation)
    
        signal = pd.read_csv(name)
        annotation = pd.read_csv(annotation)
    
        sample = name.split('/')[-1].strip()
        signal['sample_name'] = sample
        annotation['sample_name'] = sample
        
        X.append(signal)
        Y.append(annotation)
        
    X = pd.concat(X) 
    Y = pd.concat(Y)
    
    return X, Y

In [98]:
X_train, Y_train = get_data(os.path.join(DATA_FOLDER, 'scenario_1/train/physiology'))

In [99]:
X_train.head()

,time,ecg,bvp,gsr,rsp,skt,emg_zygo,emg_coru,emg_trap,sample_name
0,0,1.050,36.099,5.844,32.808,27.486,5.604,7.371,7.124,sub_1_vid_20.csv
1,1,1.053,35.659,5.839,32.789,27.493,5.645,7.370,7.083,sub_1_vid_20.csv
2,2,1.040,35.779,5.804,32.789,27.486,5.645,7.452,7.123,sub_1_vid_20.csv
3,3,1.046,35.924,5.843,32.789,27.493,5.605,7.371,7.165,sub_1_vid_20.csv
4,4,1.043,35.973,5.839,32.789,27.486,5.523,7.411,7.124,sub_1_vid_20.csv


In [100]:
Y_train.head()

,time,valence,arousal,sample_name
0,0,5.0,5.0,sub_1_vid_20.csv
1,50,5.0,5.0,sub_1_vid_20.csv
2,100,5.0,5.0,sub_1_vid_20.csv
3,150,5.0,5.0,sub_1_vid_20.csv
4,200,5.0,5.0,sub_1_vid_20.csv


In [101]:
#some complex model here -- using global mean for now
mean_valence, mean_arousal = Y_train['valence'].mean(), Y_train['arousal'].mean()

In [118]:
#predict and store
import pathlib

for path in glob(os.path.join(DATA_FOLDER, 'scenario_1/test/annotations', '*')):
    Y_test = pd.read_csv(path)
    Y_test['valence'] = mean_valence
    Y_test['arousal'] = mean_arousal
    
    directory, fname = '/'.join(path.split('/')[:-1]), path.split('/')[-1]
    directory = directory.replace('data', 'results')
    pathlib.Path(directory).mkdir(parents=True, exist_ok=True) 
    
    Y_test.to_csv(os.path.join(directory, fname), index=False)

# Scenario 2

In [119]:
def concat(path):
    
    fnames = glob(os.path.join(path, '*'))
    X, Y = [], []
    for name in fnames:
    
        annotation = name.split('/')
        annotation[-2] = 'annotations'
        annotation = '/'.join(annotation)
    
        signal = pd.read_csv(name)
        annotation = pd.read_csv(annotation)
    
        sample = name.split('/')[-1].strip()
        signal['sample_name'] = sample
        annotation['sample_name'] = sample
        
        X.append(signal)
        Y.append(annotation)
        
    X = pd.concat(X) 
    Y = pd.concat(Y)
    
    return X, Y

In [124]:
#loop across folds
for fold in glob(os.path.join(DATA_FOLDER, 'scenario_2', '*')):
    
    #train a model on non-held-out folds
    X_train, Y_train = get_data(os.path.join(fold, 'train/physiology'))
    mean_valence, mean_arousal = Y_train['valence'].mean(), Y_train['arousal'].mean()
    
    #predict on the held out fold
    for path in glob(os.path.join(fold, 'test/annotations', '*')):
        Y_test = pd.read_csv(path)
        Y_test['valence'] = mean_valence
        Y_test['arousal'] = mean_arousal
    
        directory, fname = '/'.join(path.split('/')[:-1]), path.split('/')[-1]
        directory = directory.replace('data', 'results')
        pathlib.Path(directory).mkdir(parents=True, exist_ok=True) 
    
        Y_test.to_csv(os.path.join(directory, fname), index=False)